In [ ]:
import english_words
import numpy as np
import random
import requests
import string
import time
from collections import defaultdict as dd
from copy import deepcopy as dc
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

DEVICE = "cuda:0"
MODEL_SENT = "cardiffnlp/twitter-roberta-base-sentiment"

model_sent = AutoModelForSequenceClassification.from_pretrained(MODEL_SENT).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_SENT)

WORDS = list(english_words.get_english_words_set(["web2"], alpha=True))

WLISTS = dd(list)
for word in WORDS:
    WLISTS[len(word)].append(word)

BASE_SENTS = [0.002206048, 0.03251819, 0.9652757]

def data2text(data):
    return data[0].capitalize() + " " + " ".join(data[1:]) + "!"

def get_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt").to(DEVICE)
    logits = model_sent(**tokens)
    return list(softmax(logits[0][0].cpu().detach().numpy()))

def query(data):
    text = data2text(data)
    return get_sentiment(text)

def query_remote(data):
    text = data2text(data)
    b = 0
    while True:
        try:
            res = requests.post("http://passphrase.advml.com/score", json={"data": text})
            res_json = res.json()
            if "flag" in res_json:
                print(f"'{text}'", res_json["flag"])
                return True
            return False
        except:
            b += 1
            time.sleep(b * 2)

def get_score_sent(this_sents, r=15):
    return sum((round(s0, r) - round(s1, r)) ** 2 for s0, s1 in zip(this_sents, BASE_SENTS)) ** 0.5

def get_scores(this_sents):
    return get_score_sent(this_sents), get_score_sent(this_sents, r=4)

def get_count(score):
    if nc > 2000: return 4
    elif nc > 1000: return 3
    elif nc > 100: return 2
    else: return 1

def change(this, nc):
    count = get_count(nc)
    while True:
        next = dc(this)
        for _ in range(count):
            i = random.randint(0, len(next) - 1)
            l = len(next[i])
            next[i] = random.choice(WLISTS[l])
        if next != ["my", "secret", "sauce", "is", "the", "best"]: return next

this_words = ["an", "hidden", "great", "it", "was", "good"]
this_sents = query(this_words)
best_score, best_api_score = get_scores(this_sents)

i = 0
best_flag = ""
nc = 0
while best_api_score != 0:
    i += 1
    next_words = change(this_words, nc)
    next_sents = query(next_words)
    next_score, next_api_score = get_scores(next_sents)
    if nc > 2000: mul = 8.0
    elif nc > 1000: mul = 4.0
    elif nc > 100: mul = 1.5
    else: mul = 1.1
    if next_score <= best_score * mul:
        nc = 0
        if next_score < best_score:
            best_score = next_score
            best_api_score = next_api_score
            f = query_remote(next_words)
            best_flag = data2text(next_words)
            if f: break
            print(str(i).zfill(6), f"'{best_flag}'", round(best_score, 6), round(best_api_score, 6), next_sents)
        elif round(next_api_score, 6) == 0:
            f = query_remote(next_words)
            if f: break
            print(str(i).zfill(6), f"'{data2text(next_words)}'", round(next_score, 6), round(next_api_score, 6), next_sents)
        this_words = next_words
    else:
        nc += 1

print("FOUND:", best_flag)